In [26]:
# Todo
# Try to use more of the article
# Compare to country's stock exchange
# Only American stocks
# Sentiment from 1 to 10
# Compare to close price diff according to the article time
#
# Trade
# Trading API


In [25]:
# %pip install openai yfinance --upgrade

In [1]:
import os
import json
import random
import datetime
from dateutil.relativedelta import relativedelta
from dateutil import parser
import requests
from tqdm.auto import tqdm
import pandas as pd
import numpy as np

from openai import OpenAI
import yfinance as yf

In [3]:
j = json.load(open("/root/.aws/secrets.json"))
os.environ["OPENAI_API_KEY"] = j["OPENAI_KEY"]

In [4]:
client = OpenAI()


In [5]:
def get_news(date):
    url = f"https://reuters-business-and-financial-news.p.rapidapi.com/article-date/{date}"

    headers = {
        "X-RapidAPI-Key": "4169c88d67msh97cc35168a11094p1b7c3fjsn872bf1cc2c07",
        "X-RapidAPI-Host": "reuters-business-and-financial-news.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers)
    return response.json()


In [136]:
# company_name = "coca-cola"

In [132]:
# yfinance = "https://query2.finance.yahoo.com/v1/finance/search"
# user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
# params = {"q": company_name, "quotes_count": 1, "country": "United States"}

# res = requests.get(url=yfinance, params=params, headers={'User-Agent': user_agent})
# data = res.json()

# company_code = data['quotes'][0]['symbol']


In [137]:
# company_code

In [6]:
GSPC = yf.Ticker("^GSPC")
hist = GSPC.history(start="2014-01-01")
hist['Diff'] = (hist['Close']-hist['Close'].shift(1))/hist['Close']
hist = hist.reset_index()
hist['Date'] = hist['Date'].dt.strftime('%Y-%m-%d')
snp = hist.set_index("Date")
snp.to_csv("data/s&p.txt", sep='\t')


In [6]:
snp = pd.read_csv("data/s&p.txt", sep='\t', index_col=0)

In [19]:
system_prompt = """
You are a financial advisor designed to output JSON. 
The JSON should contain the following fields: a 'stock' field, a 'ticker' field, a 'country' field, a 'sentiment' field and 'reasoning' field.
The 'stock' field is a united states traded stock related to the given news article.
The 'ticker' field is the ticker for that stock.
The 'sentiment' field is the sentiment of the article regarding the stock, it should be on a scale from 1 to 10, 1 being very negative and 10 being very positive.
The 'reasoning' field is an expanation for the given sentiment.
"""

In [8]:
def get_sentiment(data):
    response = client.chat.completions.create(
      model="gpt-4-1106-preview",
      response_format={ "type": "json_object" },
      messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"News article: {data}"},
      ]
    )

    return json.loads(response.choices[0].message.content)
    

In [78]:
NO_TICKER = {"N/A","NONE","VARIES","VARIES","NOT LISTED","VARIOUS","NOT PUBLICLY TRADED","NOT PROVIDED","MULTIPLE","UNKNOWN","PRIVATE COMPANY","PRIVATE","NOT APPLICABLE","PRIVATE","UNAVAILABLE",","}

In [38]:
def is_after_hours(article):
    article_date = article['publishedAt']['date']
    d = parser.parse(article_date)
    return d.hour>=21

In [130]:
news = get_news("2019-11-13")

In [129]:
print('hi')

hi


In [131]:
news = [n for n in news if "Friends" in n['articlesName']]

In [132]:
article = news[0]

In [133]:
article

{'articlesName': "'Friends' reunion special could be headed for HBO Max - Hollywood media",
 'articlesShortDescription': 'Could "Friends" be getting back together, if only for a one night stand?',
 'articlesDescription': '[{"type":"paragraph","content":"Could &ldquo;Friends&rdquo; be getting back together, if only for a one night stand?"},{"type":"paragraph","content":"The Hollywood Reporter and Variety on Tuesday reported that preliminary talks were underway for an unscripted reunion special that would feature all six &ldquo;Friends&rdquo; actors and air on upcoming streaming service HBO Max, a unit of AT\\u0026amp;T&rsquo;s WarnerMedia."},{"type":"paragraph","content":"HBO Max had no comment on the reports, which follow hints by Jennifer Aniston that something might be underway."},{"type":"paragraph","content":"The talks are still in the very early stages and would not involve reviving the hit comedy series that ended in 2004, the two entertainment industry publications said, citing 

In [120]:
desc = json.loads(article['articlesDescription'])

In [122]:
content = '\n'.join([x['content'] for x in desc if x['type']=='paragraph'])

In [123]:
data = f"{article['articlesName']} - {article['articlesShortDescription']} - {content}"

In [124]:
j = get_sentiment(data)

In [125]:
j

{'stock': 'Liberty Media Corporation',
 'ticker': 'FWONA',
 'country': 'United States',
 'sentiment': 8,
 'reasoning': "The article presents a positive sentiment towards Formula One's growing presence in the American market, highlighting the success of the Miami Grand Prix and the anticipation around the Las Vegas race. This optimistic outlook is likely to reflect well on Liberty Media—the company that owns Formula One—due to increased fan and sponsor interest, and the potential expansion with the Andretti Global team which indicates growth and diversity within the sport."}

In [77]:
def analyse_article(article, date):
    data = f"{article['articlesName']} - {article['articlesShortDescription']}"
    if 'United States' not in [k['keywordName'] for k in article['keywords']]:
        # print('not in US')
        return
    j = get_sentiment(data)
    if j['ticker'] in NO_TICKER or any([x in j['ticker'] for x in NO_TICKER]):
        # print('no ticker')
        return
    ticker = yf.Ticker(j['ticker'])

    # before = datetime.date(date.year,date.month,1)
    if is_after_hours(article):
        i = list(snp.index).index(date)
        start = date
        end = snp.iloc[list(snp.index).index(date)+2].name
    else:
        start = snp.iloc[list(snp.index).index(date)-1].name
        end = snp.iloc[list(snp.index).index(date)+1].name
    hist = ticker.history(start=start, end=end)
    if len(hist)==0:
        # print('no hist')
        return 

    ticker_start = hist.iloc[0]
    ticker_end = hist.iloc[-1]
    ticker_diff = (ticker_end['Close']-ticker_start['Close'])/ticker_start['Close']

    snp_start = snp.iloc[list(snp.index).index(start)]
    snp_end = snp.iloc[list(snp.index).index(end)]
    snp_diff = (snp_end['Close']-snp_start['Close'])/snp_start['Close']

    diff = ticker_diff-snp_diff
    return date, data, j['ticker'], j['sentiment'], j['reasoning'], ticker_diff, snp_diff, diff
    

In [74]:
res = []

In [ ]:
for i in tqdm(range(100)):
    date = random.choice(snp.index)
    news = get_news(date)
    for _ in range(10):
        try:
            article = random.choice(news)
            analysis = analyse_article(article,date)
            if analysis is None:
                continue

            res += [analysis]
            break
        except Exception as e:
            print(e)
            continue

  0%|          | 0/100 [00:00<?, ?it/s]

SHLDQ: No timezone found, symbol may be delisted
CTVA: Data doesn't exist for startDate = 1549429200, endDate = 1549602000


'ticker'


SPX: No price data found, symbol may be delisted (1d 2021-02-09 -> 2021-02-11)
DKNG: Data doesn't exist for startDate = 1478232000, endDate = 1478581200
WMG: Data doesn't exist for startDate = 1478232000, endDate = 1478581200
MYL: No timezone found, symbol may be delisted


argument of type 'NoneType' is not iterable


ELY: No timezone found, symbol may be delisted
PKI: No timezone found, symbol may be delisted
Failed to get ticker 'N/A' reason: Expecting value: line 1 column 1 (char 0)
N/A: No timezone found, symbol may be delisted


In [100]:
print('hi')

hi


In [101]:
df = pd.DataFrame(res) # 

In [104]:
df['sentiment'] = df[3]-5.5

In [ ]:
# df[3].apply(lambda x: x['sentiment'])

In [22]:
D = {"positive":1, 'neutral':0, 'negative':-1}

In [23]:
df['sentiment'] = df[3].apply(lambda x: D[x['sentiment'].lower()])


In [105]:
df

,0,1,2,3,4,5,6,7,sentiment
0,2016-01-15,Private equity deals hit as banks curb lending...,KKR,4,The article suggests that banks are cutting ba...,-0.036559,-0.021079,-0.015481,-1.5
1,2019-06-04,Developers sue Apple over App Store practices ...,AAPL,3,The lawsuit over App Store practices suggests ...,0.036584,0.029769,0.006815,-2.5
2,2021-07-13,UPDATE 1-Pound extends fall after U.S. inflati...,FXB,3,The pound extending its fall typically indicat...,-0.005155,-0.002356,-0.002799,-2.5
3,2016-10-25,"Caterpillar sees little change into 2017, cuts...",CAT,3,The news article indicates Caterpillar Inc has...,-0.017560,-0.005532,-0.012029,-2.5
4,2018-08-15,Google provides data on U.S. political adverti...,GOOGL,7,The addition of a section on political adverti...,-0.020602,0.000257,-0.020859,1.5
...,...,...,...,...,...,...,...,...,...
111,2020-12-04,Breakingviews - Corona Capital: Movie releases...,WBD,5,The news article discusses impacts of the pand...,0.008499,0.006884,0.001615,-0.5
112,2020-10-16,Shares of gun makers gain as Biden's lead grow...,RGR,8,The sentiments are positive towards gun makers...,-0.006867,-0.016197,0.009330,2.5
113,2019-12-03,PRESS DIGEST-British Business - Dec 3 - The fo...,TRI,5,The content relevant to the stock is not direc...,-0.007563,-0.000357,-0.007206,-0.5
114,2015-12-11,U.S. oil export ban 'very likely' to be lifted...,XOM,8,The lifting of the U.S. oil export ban is expe...,0.022733,0.015425,0.007309,2.5


In [106]:
(df[7]*df['sentiment']).sum()/np.abs(df['sentiment']).sum()

0.00741776328919238

In [32]:
df['sign'] = df[2].apply(lambda x: np.sign(x))

In [107]:
df[7].sum()/len(df)

0.003832742440477173

In [108]:
df.to_csv('data/news2roi.tsv', sep='\t', index=0)

In [67]:
print('hi')

hi


In [36]:
tp = ((df[2]>0) & (df['sentiment']>0)).sum()

In [37]:
tn = ((df[2]<0) & (df['sentiment']<0)).sum()

In [38]:
fp = ((df[2]<0) & (df['sentiment']>0)).sum()

In [39]:
fn = ((df[2]>0) & (df['sentiment']<0)).sum()

In [40]:
acc = (tp+tn)/(tp+tn+fp+fn)

In [41]:
acc

0.6

In [42]:
tp

23

In [43]:
tn

25

In [44]:
fp

23

In [45]:
fn

9